## MobileOne: An Improved One millisecond Mobile Backbone

Contents:
- [Re-parameterize](#initialize-the-model-with-pre-trained-model-weights-before-re-parameterization)
- [Inference](#model-inference-using-fused-re-parameterized-model)

In [12]:
from PIL import Image

import torch
from torchvision import transforms
from assets.meta import IMAGENET_CATEGORIES
from models import (
    mobileone_s0,
    mobileone_s1,
    mobileone_s2,
    mobileone_s3,
    mobileone_s4,
    reparameterize_model
)

#### Initialize the model with pre-trained model weights (before re-parameterization)

In [3]:
model = mobileone_s0(pretrained=True)

INFO: Pre-trained weights successfully loaded.


In [4]:
model.eval() # change to evaluation mode
reparam_model = reparameterize_model(model)  # re-parameterize model

In [5]:
torch.save(reparam_model.state_dict(), "weights/s0_fused.pt") # save re-parameterized model

### Model Inference using fused (re-parameterized) model

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = mobileone_s0(inference_mode=True) # initialize the model in inference mode

# load re-parameterized model weights
model.load_state_dict(torch.load("weights/s0_fused.pt", map_location=device, weights_only=True))

INFO: Creating model without pre-trained weights.


<All keys matched successfully>

In [7]:
def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize the image to match the model's input size
        transforms.ToTensor(),  # Convert the image to a PyTorch tensor
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],  # Normalize the image using the mean and std of ImageNet
            std=[0.229, 0.224, 0.225]
        ),
    ])

    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)  # Add a batch dimension
    return image

In [15]:
def inference(model, image_path):
    model.eval()

    input_image = preprocess_image(image_path)
    with torch.no_grad():
        output = model(input_image)

    # Get the top 5 predictions and their confidence scores
    probabilities = torch.nn.functional.softmax(output, dim=1)
    top5_prob, top5_catid = torch.topk(probabilities, 5)

    for i in range(top5_prob.size(1)):
        predicted_label = IMAGENET_CATEGORIES[top5_catid[0][i].item()]
        confidence = top5_prob[0][i].item()
        print(f"Predicted class label: {predicted_label}, Confidence: {confidence:.4f}")

inference(model, "assets/tabby_cat.jpg")

Predicted class label: tabby, Confidence: 0.8457
Predicted class label: tiger cat, Confidence: 0.1194
Predicted class label: Egyptian cat, Confidence: 0.0275
Predicted class label: lynx, Confidence: 0.0007
Predicted class label: printer, Confidence: 0.0003
